In [8]:
import QuantLib as ql
 
import datetime 
from datetime import datetime

risk= [[[8, 11, 2022], [16, 12, 2022], 50, 51, 0.294, 0.02, [[[8, 11, 2022], 0]], True, True]]

def get_option_price(
    valuation_date = [8, 11, 2022],
    expiry_date = [16, 12, 2022],
    underlying_price = 50,
    strike_price = 51,
    volatility =  0.294,
    risk_free_rate = 0.02,
    dividends = [[[8, 11, 2022], 0]],
    is_american = True,
    is_call = True
    ):
    
    div_dates = []
    div_values = []
    if (dividends==0):
        dividend=0
    else:
        for div in dividends:
            date = div[0]
            value = div[1]
            div_values.append(value)
            div_dates.append(ql.Date(*date))
          
    
    # Reformat dates from list into QL date format
    valuation_date = ql.Date(*valuation_date)
    expiry_date = ql.Date(*expiry_date)
    ql.Settings.instance().setEvaluationDate(valuation_date)
    day_count = ql.Actual365Fixed()
    calendar = ql.UnitedStates()
    
    # Reformat prices and rates from list into QL format
    underlying_price = ql.QuoteHandle(ql.SimpleQuote(underlying_price))
    risk_free_rate = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, risk_free_rate, day_count))
    volatility = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(valuation_date, calendar, volatility, day_count))
    
    # Create option
    if is_call:
        payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike_price)
    else:
        payoff = ql.PlainVanillaPayoff(ql.Option.Put, strike_price)
    if is_american:
        exercise = ql.AmericanExercise(valuation_date, expiry_date)
    else:
        exercise = ql.EuropeanExercise(expiry_date)
    option = ql.DividendVanillaOption(payoff, exercise, div_dates,
                                          div_values)
    
    # Black Scholes process
    process = ql.BlackScholesProcess(underlying_price,
                                          risk_free_rate,
                                          volatility)
    
    # Create option's pricing engine
    precision_steps = 1000
    engine = ql.FdBlackScholesVanillaEngine(process, precision_steps, precision_steps - 1)
    option.setPricingEngine(engine)
    print(option.NPV())
    # Price the option
    return option.NPV()
   

In [9]:
price = get_option_price(is_call = True )
print("Call with dividend",price)

1.4946461002889466
Call with dividend 1.4946461002889466


In [3]:
price = get_option_price( is_call = False )
print("Call with dividend",price)

2.5772697119960615
Call with dividend 2.5772697119960615


In [4]:
price = "{0:.3f}".format(get_option_price(dividends = 0, is_call = True))
print("Call with zero dividend",price)

2.2638838535022283
Call with zero dividend 2.264


In [5]:
price = "{0:.3f}".format(get_option_price(dividends = 0, is_call = False))
print( "Put with zero dividend",price)

2.3869080351184957
Put with zero dividend 2.387


In [ ]:
import numpy as np
from scipy.stats import norm
    # Without dividend
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset
N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)
        

In [ ]:
BS_CALL(50, 100, 1, 0.05, 0.25)

In [ ]:
import numpy as np
from scipy.stats import norm
    # With dividend
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset
N = norm.cdf

def BS_CALLDIV(S, K, T, r, q, sigma):
    d1 = (np.log(S/K) + (r - q + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return S*np.exp(-q*T) * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUTDIV(S, K, T, r, q, sigma):
    d1 = (np.log(S/K) + (r - q + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*np.exp(-q*T)*N(-d1)

In [ ]:
BS_PUTDIV(50, 100, 1, 0.05,0.05, 0.25)